# Introduction to the PIC-SURE API

This is a tutorial notebook aimed to get the user quickly up and running with the R PIC-SURE API. 

## PIC-SURE R API
### What is PIC-SURE?

As part of the *NHLBI BioData Catalyst® (BDC)* ecosystem, the Patient Information Commons Standard Unification of Research Elements (PIC-SURE) platform has been integrating clinical and genomic datasets from multiple TOPMed and TOPMed related studies funded by the National Heart Lung and Blood Institute (NHLBI).

Original data exposed through PIC-SURE API encompasses a large heterogeneity of data organization underneath. PIC-SURE hides this complexity and exposes the different study datasets in a single tabular format. By simplifying the process of data extraction, it allows investigators to focus on the downstream analyses and to facilitate reproducible science.


### More about PIC-SURE
The API is available in two different programming languages, python and R, enabling investigators to query the databases the same way using either language.


PIC-SURE is a larger project from which the R/python PIC-SURE API is only a brick. Among other things, PIC-SURE also offers a graphical user interface that allows researchers to explore variables across multiple studies, filter patients that match criteria, and create cohorts from this interactive exploration.

The R API is actively developed by the Avillach Lab at Harvard Medical School.

PIC-SURE API GitHub repo:
* https://github.com/hms-dbmi/pic-sure-r-adapter-hpds
* https://github.com/hms-dbmi/pic-sure-r-client
* https://github.com/hms-dbmi/pic-sure-biodatacatalyst-r-adapter-hpds




 -------

## Getting your own user-specific security token

**Before running this notebook, please be sure to review the "Get your security token" documentation, which exists in the [`README.md` file](../README.md). It explains how to get a security token, which is mandatory to use the PIC-SURE API.**

To set up your token file, be sure to run the [`Workspace_setup.ipynb` file](./Workspace_setup.ipynb).

### Environment set-up

#### Pre-requisites
- R 3.4 or later

#### Install packages

The first step to using the PIC-SURE API is to install the packages needed. The following code installs the PIC-SURE API components from GitHub, specifically:

- PIC-SURE Client
- PIC-SURE Adapter
- *BDC-PIC-SURE* Adapter

**Note that if you are using the dedicated PIC-SURE environment within the *BDC Powered by Seven Bridges (BDC-Seven Bridges)* platform, the necessary packages have already been installed.**

*Troubleshooting: if you encounter an error while installing packages or connecting to a PIC-SURE resource, try restarting your kernel and trying again.*

In [ ]:
### Uncomment this code if you are not using the PIC-SURE environment in *BDC-Seven Bridges*, or if you do not have all the necessary dependencies installed.
#install.packages("devtools")

In [ ]:
Sys.setenv(TAR = "/bin/tar")
options(unzip = "internal")
devtools::install_github("hms-dbmi/pic-sure-r-adapter-hpds", ref="main", force=T, quiet=FALSE)
library(dplyr)

## Connecting to a PIC-SURE resource

The following is required to get access to data through the PIC-SURE API: 
- Network URL
- User-specific security token

The following code specifies the network URL as the *BDC-PIC-SURE* URL and references the user-specific token saved as `token.txt`.

If you have not already retrieved your user-specific token, please refer to the "Get your security token" section of the `README.md` file and the `Workspace_setup.ipynb` file.

In [ ]:
PICSURE_network_URL = "https://picsure.biodatacatalyst.nhlbi.nih.gov/picsure"
token_file <- "token.txt"
token <- scan(token_file, what = "character")
session <- picsure::bdc.initializeSession(PICSURE_network_URL, token)
session <- picsure::bdc.setResource(session = session,  resourceName = "AUTH")

### Getting help with the PIC-SURE API

You can get help with PIC-SURE library functions by using the `?` operator

In [ ]:
?picsure::bdc.searchPicsure

For example, the above output lists and briefly defines how to use the `searchPicsure` function.

## Using the PIC-SURE variable dictionary

Now that you have set up your connection to the PIC-SURE API, let's determine which study or studies you are authorized to access. The `searchPicsure` function can be used to search the data dictionary for a specific term or to retrieve information about all the variables you are authorized to access. 


In [ ]:
searchResults <- picsure::bdc.searchPicsure(session) # retrieve all the variables you have access to

In [ ]:
studies <- unique(searchResults$study_id)

In [ ]:
print("You are authorized to access the following studies:")
print(studies)

For the rest of this example notebook, we will use one of the publicly available datasets available on PIC-SURE. This dataset is the "Digitalis Investigation Group (DIG) Trial: Dataset for Teaching Purposes", which is listed as `tutorial-biolincc_digitalis` in the PIC-SURE output above.

In [ ]:
phs_number <- "tutorial-biolincc_digitalis"

Now, let's find all of the variables associated with that study. We can search for these using the `searchPicsure` function and searching the phs accession number; in this case `tutorial-biolincc_digitalis`. We are interested in the values associated with the categorial variables in this study, so we will make sure to include the argument `includeValues = TRUE`.

In [ ]:
my_variables_df <- picsure::bdc.searchPicsure(session, keyword = phs_number, includeValues = TRUE) # Search for the phs accession number 

We can now review the variables that were returned and some of the information associated with these variables.

In [ ]:
nrow(my_variables_df) # Count number of rows to determine number of variables that matched our search

In [ ]:
head(my_variables_df$var_description, 10) # Review the first 10 variable descriptions in the dataframe

PIC-SURE integrates clinical and genomic datasets across *BDC*, including TOPMed and TOPMed-related studies, COVID-19 studies, and BioLINCC studies. Each variable is organized as a concept path that contains information about the study, variable group, and variable. Though the specifics of the concept paths are dependent on the type of study, the overall information included is the same. 

Data Organization in PIC-SURE
---------------------------------------
| Data organization | TOPMed & TOPMed-related studies | BioLINCC & COVID-19 studies |
|-------------------|---------------------------------|-----------------------------|
| General organization | Data organized using the format implemented by the database of Genotypes and Phenotypes (dbGaP). Generally, a given study will have several tables, and those tables have several variables. | Data do not follow dbGaP format; there are no phv or pht accessions. Data are organized in groups of like variables, when available. For example, variables like Age, Gender, and Race could be part of the Demographics variable group. |
| Concept path structure | \phs\pht\phv\variable name\ | \phs\variable name |
| Variable ID | phv corresponding to the variable accession number | Equivalent to variable name | 
| Variable name | Encoded variable name that was used by the original submitters of the data | Encoded variable name that was used by the original submitters of the data |
| Variable description | Description of the variable | Description of the variable, as available |
| Dataset ID | pht corresponding to the trait table accession number | Equivalent to Dataset name | 
| Dataset name | Name of the trait table | Name of a group of like variables, as available | 
| Dataset description | Description of the trait table | Description of a group of like variables, as available |
| Study ID | phs corresponding to the study accession number | phs corresponding to the study accession number |
| Study description | Description of the study from dbGaP | Description of the study from dbGaP |



Now you can try to search for a term on your own. Below is sample code on how to search for the term `sex`. To practice searching the data dictionary, you can change "sex" to a term you are interested in. Note - the results displayed will show results from all studies you have access to. 

In [ ]:
my_search_df <- picsure::bdc.searchPicsure(session, 'sex') # Change sex to be your term of interest
head(my_search_df)

## Using PIC-SURE to build a query and retrieve data
You can also use the PIC-SURE API to build a query and retrieve data. With this functionality, you can filter based on specific variables, add others, and export the data as a dataframe into this notebook. 

The first step is to set up the query object.

In [ ]:
query_categorical_example <- picsure::bdc.newQuery(session)

There are several methods that can be used to build a query, which can be specified by the `type` argument in the `picsure::addClause` function.

| type | functionality |
|--------|-------------------|
| type = 'SELECT' | return all variables included in the list (no record subsetting)|
| type = 'REQUIRE' | return all variables included in the list; only records that do not contain null values for input variables |
| type = 'ANYOF' | return all variables included in the list; only records that contain at least one non-null value for input variables |
| type = 'FILTER' | return input variable; only records that match filter criteria |

### Build a query with a categorical variable
Let's practice building a query by filtering on variables. First, let's select a categorical variable to use. We can identify one using the `is_categorical` column of the variable dataframe.

In [ ]:
categorical_vars <- my_variables_df[my_variables_df$data_type == 'categorical',] # Filter to only categorical variables
head(categorical_vars$var_description) #View the first 5 examples
#categorical_vars # Uncomment to view the full dataframe

Let's do a simple filter on the `SEX` variable to select only females.

In [ ]:
#categorical_vars[categorical_vars$var_description == "SEX", ] # Uncomment to view information about the variable

In [ ]:
query_categorical_example <- picsure::addClause(query = query_categorical_example, # Query to build on
                                                keys = "\\tutorial-biolincc_digitalis\\SEX\\", # Variable concept path to filter on
                                                type = "FILTER", # The type of query, in this case a FILTER
                                                categories = list("Female")) # Value(s) to filter on

Note that though we are only filtering by one value, you can filter by multiple values by passing a list into the `filter` function.

Now we can export our filtered data to a dataframe in this notebook.

In [ ]:
results_categorical <- picsure::runQuery(query_categorical_example, resultType = 'DATA_FRAME')
head(results_categorical)

In the data dictionary dataframe shown previously, each row represented a single concept path or variable. In the query dataframe, the concept paths are added as columns with each row representing a participant with data that matches your query. 

The dataframe above should contain some automatically exported concept paths, such as `Patient ID`, `Parent Study Accession with Subject ID`, `Topmed Study Accession with Subject ID`, and `consents`, and the concept path we added to our query (`\tutorial-biolincc_digitalis\SEX\`). Additionally, all participants should have the value we used to filter for our added concept path.

We can see how this query filtering worked by comparing the resulting dataframe to the full unfiltered data for this variable. Let's build a query that retrieves the data from all participants that have data for the categorical variable (in this case, both males and females) of interest using `require()`.

In [ ]:
full_categorical_example <- picsure::bdc.newQuery(session) #Initialize a new query
full_categorical_example <- picsure::addClause(query = full_categorical_example,
                                               keys = "\\tutorial-biolincc_digitalis\\SEX\\",
                                               type = "REQUIRE")

full_results_categorical <- picsure::runQuery(full_categorical_example)
head(full_results_categorical)

In [ ]:
# Visualize the results with pie charts
df_full <- data.frame(table(full_results_categorical[,eval("\\tutorial-biolincc_digitalis\\SEX\\")]))
pie(df_full$Freq, labels = df_full$Var1, main = paste("Before filtering on SEX"))

df_filter <- data.frame(table(results_categorical[, eval("\\tutorial-biolincc_digitalis\\SEX\\")]))
pie(df_filter$Freq, labels = df_filter$Var1, main = paste("After filtering on SEX"))

### Build a query with a continuous variable
Similarly, we can create a query using a continuous variable. Let's first filter to continuous variables using the `data_type` column.

In [ ]:
continuous_vars <- my_variables_df[my_variables_df$data_type == 'continuous',] # Filter to only categorical variables
head(continuous_vars$var_description) #View the first 5 examples
#continuous_vars # Uncomment to view the full dataframe

Let's do a simple filter on age to include the age range from 30 to 50 years old.

In [ ]:
#continuous_vars[continuous_vars$var_name == "AGE", ] # Uncomment to view information about the variable

In [ ]:
query_continuous_example <- picsure::bdc.newQuery(session) # Initialize a new query
query_continuous_example <- picsure::addClause(query = query_continuous_example, # Query to build on
                                               keys = "\\tutorial-biolincc_digitalis\\AGE\\", # Variable concept path to filter on
                                               type = "FILTER", # The type of query, in this case FILTER
                                               min = 30, # Minimum of the range
                                               max = 50) # Maximum of the range

Note that if you do not specify the `min` or `max` fields, the minimum or maximum of the variable will be used.

Now we can export our filtered data to a dataframe in this notebook. 

In [ ]:
results_continuous <- picsure::runQuery(query_continuous_example, resultType = 'DATA_FRAME')
head(results_continuous)

We can see how this query filtering worked by comparing the resulting dataframe to the full unfiltered data for this variable. Let's build a query that retrieves the data from all participants that have data for the continuous variable (in this case, participants of all ages) of interest using `require()`.

In [ ]:
full_continuous_example <- picsure::bdc.newQuery(session) #Initialize a new query
full_continuous_example <- picsure::addClause(query = full_continuous_example,
                                               keys = "\\tutorial-biolincc_digitalis\\AGE\\",
                                               type = "REQUIRE")

full_results_continuous <- picsure::runQuery(full_continuous_example)
head(full_results_continuous)

In [ ]:
# Visualize results with histograms
hist(full_results_continuous[,eval("\\tutorial-biolincc_digitalis\\AGE\\")],
     main = "Before filtering on AGE",
     xlab = "Age (in years)")

In [ ]:
hist(results_continuous[,eval("\\tutorial-biolincc_digitalis\\AGE\\")], 
     main = "After filtering on AGE",
     xlab = "Age (in years)")

### Build a query with multiple variables
You can also add multiple variables to a single query. Let's build a query with the first five variables for the study of interest.

In [ ]:
query_vars_readable <- my_variables_df[c(5:10), "var_description"]
query_vars_concept_path <- my_variables_df[c(5:10), "name"]
query_vars_readable

We can use the`anyof` function to add variables to the query. This will filter to participants that have data **for at least one of the variables added**.  

In [ ]:
query_anyof <- picsure::bdc.newQuery(session) # Initialize a new query
query_anyof <- picsure::addClause(query = query_anyof, 
                                keys = query_vars_concept_path,
                                type = 'ANYOF') 
results <-  picsure::runQuery(query_anyof)

In [ ]:
head(results)

### Selecting consent groups

PIC-SURE will limit results based on which study and consent groups you have been individually authorized to access. In some cases, such as instances where you can access multiple studies and/or consent groups, you may need to limit your results further to only a subset of the groups you have been authorized to access.

Let's see the studies and consent groups you are authorized to access by navigating to the 'consents' portion of the query object.

In [ ]:
authQuery_consents <- picsure::bdc.newQuery(session) # Initialize a new query
authQuery_consents$categoryFilters$`\\_consents`

The output shown above lists all of the phs accession numbers and consent codes that you are authorized to access. 

To query on specific consent groups in this list, you must first clear the list of values within the `\\_consents\\` section and then manually replace them. Let's practice this by copying and pasting a phs accession number and consent code, deleting the `\\_consents\\` field, and adding it back with our selected consent code. We can use another publicly available dataset in PIC-SURE, "Childhood Asthma Management Program (CAMP: Dataset for Teaching Purposes" with consent code `tutorial-biolincc_camp`.

*Note that trying to manually add a consent group which you are not authorized to access will results in errors downstream.*

In [ ]:
authQuery_consents <- picsure::deleteClause(query = authQuery_consents, key = '\\_consents\\')
authQuery_consents$categoryFilters$`\\_consents`

In [ ]:
consent_group_filter <- "tutorial-biolincc_camp"
authQuery_consents <- picsure::bdc.addClause(query = authQuery_consents, 
                                         keys = '\\_consents\\', 
                                         categories = list(consent_group_filter), 
                                         type = 'FILTER')
authQuery_consents$categoryFilters$`\\_consents`

Now your query is set to select only variables and participants from `tutorial-biolincc_camp`. From here, you can build out your query as shown above.